# Часть 2

In [ ]:
import pandas as pd
import pandasql as ps

In [3]:
pd.__version__

'0.23.4'

In [5]:
android_devices = pd.read_csv('../AndroidML/android_devices.csv')
user_device = pd.read_csv('../AndroidML/user_device.csv')
user_usage = pd.read_csv('../AndroidML/user_usage.csv')

In [6]:
android_devices.head()

,Retail Branding,Marketing Name,Device,Model
0,NaN,NaN,AD681H,Smartfren Andromax AD681H
1,NaN,NaN,FJL21,FJL21
2,NaN,NaN,T31,Panasonic T31
3,NaN,NaN,hws7721g,MediaPad 7 Youth 2
4,3Q,OC1020A,OC1020A,OC1020A


In [7]:
user_device.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [8]:
user_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


## Сравнение JOIN

In [30]:
%%timeit
result1 = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id', 
                 how='left')

4.42 ms ± 163 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
result1.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


### То же самое на pandasql

In [33]:
%%timeit
result2 = ps.sqldf("""SELECT A.*, B.user_id, B.platform, B.device 
FROM user_usage AS A
LEFT JOIN user_device AS B
ON A.USE_ID=B.USE_ID""",globals())

26.4 ms ± 1.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
result2.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,user_id,platform,device
0,21.97,4.82,1557.33,22787,12921.0,android,GT-I9505
1,1710.08,136.88,7267.55,22788,28714.0,android,SM-G930F
2,1710.08,136.88,7267.55,22789,28714.0,android,SM-G930F
3,94.46,35.17,519.12,22790,29592.0,android,D2303
4,71.59,79.26,1557.33,22792,28217.0,android,SM-G361F


#### Таким образом PANDASQL сработал в 6 раз медленее чем PANDAS на джойнах

## Сравнение GROUP BY

In [59]:
%%timeit
result11 = result1.astype(str).groupby("platform")['platform'].count()

2.75 ms ± 141 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [56]:
result11

platform
android    157
ios          2
nan         81
Name: platform, dtype: int64

In [60]:
%%timeit
result21 = ps.sqldf('''SELECT count(*), platform
        FROM result2
        GROUP BY platform
    ''',globals())

18.4 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [58]:
result21

,count(*),platform
0,81,None
1,157,android
2,2,ios


#### Таким образом PANDASQL сработал вновь в 6 раз медленее чем PANDAS на группировке